### Train on the CDR dataset

In [1]:
import argparse
import sys
import os
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer
from model_bio import Model
from utils import set_seed
from prepro_bio import read_bio
from save_result import Logger
from evaluation import train, evaluate

parser = argparse.ArgumentParser()
parser.add_argument("--task", default="cdr", type=str)
parser.add_argument("--data_dir", default="./dataset/cdr", type=str)
parser.add_argument("--transformer_type", default="bert", type=str)
parser.add_argument("--model_name_or_path", default="bert-base-cased", type=str)
parser.add_argument("--train_file", default="Train.BioC.JSON", type=str)
parser.add_argument("--dev_file", default="Dev.BioC.JSON", type=str)
parser.add_argument("--test_file", default="Test.BioC.JSON", type=str)
parser.add_argument("--save_path", default="out", type=str)
parser.add_argument("--load_path", default="", type=str)
parser.add_argument("--config_name", default="", type=str,
                    help="Pretrained config name or path if not the same as model_name")
parser.add_argument("--tokenizer_name", default="", type=str,
                    help="Pretrained tokenizer name or path if not the same as model_name")
parser.add_argument("--max_seq_length", default=1024, type=int,
                    help="The maximum total input sequence length after tokenization. Sequences longer "
                         "than this will be truncated, sequences shorter will be padded.")
parser.add_argument("--train_batch_size", default=4, type=int, help="Batch size for training.")
parser.add_argument("--test_batch_size", default=8, type=int, help="Batch size for testing.")
parser.add_argument("--gradient_accumulation_steps", default=1, type=int,
                    help="Number of updates steps to accumulate before performing a backward/update pass.")
parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
parser.add_argument("--adam_epsilon", default=1e-6, type=float, help="Epsilon for Adam optimizer.")
parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
parser.add_argument("--warmup_ratio", default=0.06, type=float, help="Warm up ratio for Adam.")
parser.add_argument("--num_train_epochs", default=30.0, type=float, help="Total number of training epochs to perform.")
parser.add_argument("--evaluation_steps", default=-1, type=int, help="Number of training steps between evaluations.")
parser.add_argument("--seed", type=int, default=66, help="random seed for initialization")
parser.add_argument("--num_class", type=int, default=97, help="Number of relation types in dataset.")
parser.add_argument('--gnn', type=str, default='GCN', help="GCN/GAT")
parser.add_argument('--use_gcn', type=str, default='tree', help="use gcn, both/mentions/tree/false")
parser.add_argument('--dropout', type=float, default=0.5, help="0.0/0.2/0.5")
parser.add_argument('--loss', type=str, default='BSCELoss',
                    help="use BSCELoss/BalancedLoss/ATLoss/AsymmetricLoss/APLLoss")
parser.add_argument('--s0', type=float, default=0.3)
parser.add_argument("--demo", type=str, default='false', help='use a few data to test. default true/false')
parser.add_argument("--unet_in_dim", type=int, default=3, help="unet_in_dim.")
parser.add_argument("--unet_out_dim", type=int, default=256, help="unet_out_dim.")
parser.add_argument("--down_dim", type=int, default=256, help="down_dim.")
parser.add_argument("--bert_lr", default=3e-5, type=float, help="The initial learning rate for Adam.")
parser.add_argument("--max_height", type=int, default=64, help="max_height.")
parser.add_argument("--rel2", type=int, default=0, help="")
parser.add_argument("--save_result", type=str, default="", help="save predict result.")
args, _ = parser.parse_known_args()

if args.task == 'cdr':
    args.data_dir = './dataset/cdr'
    args.train_file = 'train_filter.data'
    args.dev_file = 'dev_filter.data'
    args.test_file = 'test_filter.data'
    args.model_name_or_path = '/data/pretrained/scibert_scivocab_cased'
    args.train_batch_size = 12
    args.test_batch_size = 12
    args.learning_rate = 2e-5
    args.num_class = 2
    args.num_train_epochs = 30

if not os.path.exists(args.save_path):
    os.mkdir(args.save_path)
file_name = "{}_{}_{}_seed_{}_{}_{}_{}_{}".format(
    args.train_file.split('.')[0],
    args.transformer_type, args.data_dir.split('/')[-1],
    args.loss, args.use_gcn, args.s0, args.dropout, str(args.seed), )
args.save_path = os.path.join(args.save_path, file_name)
args.save_pubtator = os.path.join('./result/' + args.task + '/' + args.task  + '_' + args.loss
                                  + '_' + str(args.use_gcn) + '_s0=' + str(args.s0)
                                  + '_dropout=' + str(args.dropout) + '_' + str(args.seed))
if args.load_path == "":
    sys.stdout = Logger(stream=sys.stdout, filename=args.save_pubtator + '_test.log')
read = read_bio
print(args)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.device = device

config = AutoConfig.from_pretrained(
    args.config_name if args.config_name else args.model_name_or_path, num_labels=args.num_class, )
tokenizer = AutoTokenizer.from_pretrained(
    args.tokenizer_name if args.tokenizer_name else args.model_name_or_path, )
model = AutoModel.from_pretrained(
    args.model_name_or_path, from_tf=bool(".ckpt" in args.model_name_or_path), config=config, )
config.cls_token_id = tokenizer.cls_token_id
config.sep_token_id = tokenizer.sep_token_id
config.transformer_type = args.transformer_type
set_seed(args)
model = Model(args, config, model, num_labels=1)
model.to(0)

# Training
train_file = os.path.join(args.data_dir, args.train_file)
dev_file = os.path.join(args.data_dir, args.dev_file)
test_file = os.path.join(args.data_dir, args.test_file)
train_features = read(args, train_file, tokenizer, max_seq_length=args.max_seq_length)
dev_features = read(args, dev_file, tokenizer, max_seq_length=args.max_seq_length)
test_features = read(args, test_file, tokenizer, max_seq_length=args.max_seq_length)
train(args, model, train_features, dev_features, test_features)

print("BEST TEST")
model.load_state_dict(torch.load(args.save_path + '_best'))
dev_score, dev_output = evaluate(args, model, dev_features, tag="dev")
print(dev_output)
test_score, test_output = evaluate(args, model, test_features, tag="test", generate=True)
print(test_output)

Namespace(adam_epsilon=1e-06, bert_lr=3e-05, config_name='', data_dir='./dataset/cdr', demo='false', dev_file='dev_filter.data', down_dim=256, dropout=0.5, evaluation_steps=-1, gnn='GCN', gradient_accumulation_steps=1, learning_rate=2e-05, load_path='', loss='BSCELoss', max_grad_norm=1.0, max_height=64, max_seq_length=1024, model_name_or_path='/data/pretrained/scibert_scivocab_cased', num_class=2, num_train_epochs=30, rel2=0, s0=0.3, save_path='out/train_filter_bert_cdr_seed_BSCELoss_tree_0.3_0.5_66', save_pubtator='./result/cdr/cdr_BSCELoss_tree_s0=0.3_dropout=0.5_66', save_result='', seed=66, task='cdr', test_batch_size=12, test_file='test_filter.data', tokenizer_name='', train_batch_size=12, train_file='train_filter.data', transformer_type='bert', unet_in_dim=3, unet_out_dim=256, use_gcn='tree', warmup_ratio=0.06)


epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Total steps: 1230
Warmup steps: 73
training risk: 10.586490631103516    step: 40
dev_score:  0.6418147103574903 dev_output:  {'dev_F1': 64.18147103574903, 'dev_P': 88.96672348569662, 'dev_R': 50.19762796247404}
test_score:  0.6150989700815321 test_output:  {'test_F1': 61.50989700815322, 'test_P': 87.67360958899984, 'test_R': 47.373357904565125}


epoch:   3%|▎         | 1/30 [01:19<38:13, 79.07s/it]

training risk: 10.249295234680176    step: 81
dev_score:  0.8171876429947745 dev_output:  {'dev_F1': 81.71876429947746, 'dev_P': 90.92009575157269, 'dev_R': 74.2094854327126}
test_score:  0.8008402904989914 test_output:  {'test_F1': 80.08402904989914, 'test_P': 91.65658897634113, 'test_R': 71.10694117160467}


...

training risk: 9.840618133544922    step: 1188
dev_score:  0.8708036699612896 dev_output:  {'dev_F1': 87.08036699612896, 'dev_P': 86.90944796348968, 'dev_R': 87.25296356469404}
test_score:  0.8720880202610122 test_output:  {'test_F1': 87.20880202610121, 'test_P': 90.18035981783207, 'test_R': 84.42776656259132}


epoch:  97%|█████████▋| 29/30 [38:27<01:18, 78.73s/it]

training risk: 9.796359062194824    step: 1229
dev_score:  0.8711061025374628 dev_output:  {'dev_F1': 87.11061025374627, 'dev_P': 87.06811365184488, 'dev_R': 87.15414933642145}
test_score:  0.8710412272815966 test_output:  {'test_F1': 87.10412272815967, 'test_P': 90.49544903442418, 'test_R': 83.95872341502137}


epoch: 100%|██████████| 30/30 [39:46<00:00, 79.54s/it]


BEST TEST
{'dev_F1': 87.46127331089403, 'dev_P': 90.3157885229917, 'dev_R': 84.78260785787937}
generate predict result in ./result/cdr/cdr_BSCELoss_tree_s0=0.3_dropout=0.5_66
./result/cdr/cdr_BSCELoss_tree_s0=0.3_dropout=0.5_66.pubtator
{'test_F1': 86.7524890933107, 'test_P': 92.46284402905685, 'test_R': 81.70731630668558}
